# Introduction to Geospatial data and Google Earth Engine

## What is this notebook about?

To study physical domains like river water quality, we need to think geospatially. This notebook introduces **Google Earth Engine (GEE)**, a powerful, free cloud platform built for geospatial data science. 🌎 GEE provides access to a massive public data catalog and allows us to perform large-scale analysis by combining this information with our own datasets.

In this notebook, we'll demonstrate the basic functionalities of GEE using data from our hackathon. We will visualize the Amstelland catchment and key locations within it, such as water quality sampling points and the pumps vital to the region's water management. We will also perform some basic geospatial analysis to understand how these features relate to each other.

## Basics of Geospatial data


Before starting our analysis, it’s helpful to understand the basics of how geospatial data is structured. In our field, we work with two primary types: **raster** and **vector**.

<u>RASTER DATA</u>   
Think of raster data as a digital photograph or a grid of pixels. Each pixel has a specific location and contains a value representing something, like temperature, elevation, or land cover. **Satellite imagery** is the most common form of raster data.

<u>VECTOR DATA</u>   
Think of vector data as drawing specific features on a map. It uses precise geometric shapes to represent the world:
* **Points**: Single locations, like a measurement station or a pump.
* **Lines**: Linear features, like a river or a road.
* **Polygons**: Enclosed areas, like a lake or a river catchment.

This notebook will focus primarily on working with vector data to understand the features within our study area.

## Preliminary setup: libraries, authentication, and link to Google Drive

In this tutorial, we'll use a stack of powerful open-source libraries to handle our data and create visualizations. Here are the key players:

* **earthengine-api**: The official Google library that allows us to communicate directly with the Earth Engine platform.
* **geemap**: A user-friendly library built on top of the GEE API that makes interactive mapping and analysis much easier, especially in notebooks.
* **geopandas** & **shapely**: If you've used the popular **pandas** library for table data, you'll feel right at home with **geopandas**. It extends pandas by adding a special "geometry" column, turning your data tables into powerful geospatial layers. It's the standard toolkit for vector data (points, lines, and polygons), with **shapely** working behind the scenes to handle the geometric operations.
* **ipywidgets** & **ipyleaflet**: The foundation for all the interactive map widgets and controls you'll see in this notebook.

Together, these tools provide a complete environment for geospatial data science, from analyzing global satellite data to creating rich, interactive maps of our local datasets.

In [ ]:
!pip install --quiet earthengine-api geemap geopandas shapely ipywidgets ipyleaflet

Next, we need to authenticate our session to use Google Earth Engine. This step securely connects the notebook to your GEE account.

Follow the link that will appears, sign in with your Google account, and paste the authorization code into the input box that appears below. 🔑

In [ ]:
!earthengine authenticate           # run once in a terminal / notebook

E0000 00:00:1756211834.855041   61975 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756211834.884244   61975 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756211834.994473   61975 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756211834.994577   61975 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756211834.994588   61975 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756211834.994596   61975 computation_placer.cc:177] computation placer already registered. Please check linka

Now, we connect to Google Earth Engine.

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

Lastly, let's connect this Colab notebook to your Google Drive, where our data is stored.

Run the cell below, click the authorization link, sign in with your Google account, and paste the code into the input box that appears. 📁

In [ ]:
from google.colab import drive, output
drive.mount('/content/drive')      # grants notebook access to Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

## Analyses

### Part I: visualizing the Amstel shapefile in the Amstelland region

We'll begin our analysis by displaying the **shapefile** that defines the Amstelland river system. This will visualize the geometry of the river, giving us a clear view of our area of interest. A shapefile (.shp) is a popular file format for storing geospatial vector data like the points, lines, and polygons we discussed earlier. 🗺️

First, we load the shapefile using `geopandas`. If you are familiar with `pandas`, you can see that the resulting geodataframe is essentially a dataframe with a `geometry` column.

In [ ]:
import os
import geopandas as gpd
from pathlib import Path

# --- Load Shapefile from Google Drive ---
# Define the path to the folder containing our shapefile components.
drive_folder = Path('/content/drive/MyDrive/Waternet/Hackathon/data')  # <-- change as needed
shp_path     = drive_folder / 'shapefiles/GEE (google earth engine)/amstelland_shapefile/OWMIDENT_NL11_1_1_clean.shp'

# Read the shapefile from Drive into a GeoPandas GeoDataFrame.
amstel_gdf = gpd.read_file(shp_path)
amstel_gdf.head()

,OBJECTID,OWMIDENT,OWAIDENT,OWANAAM,OWMNAAM,GELDIG_B,GELDIG_E,WBHCODE,WBHNAAM,SCHAAL,...,OWMTYPE,OWMTYPED,OWMLENGT,OWMOPPVL,SHAPE_LENG,OWMKARAK,NAMESPAC,SHAPE_STAr,SHAPE_STLe,geometry
0,2.0,NL11_1_1,NL11_2000-EAG-1,"Boezem Amstelland-West, Noord",Amstellandboezem,None,None,NL_W11,"Waterschap Amstel, Gooi en Vecht",50000.0,...,M6b,R6,NaN,5.192,NaN,"Groot, ondiep kanaal dat vooral bestaat uit op...",11.0,924536.990,50872.806,"MULTIPOLYGON (((123613.27 486583.824, 123615.2..."
1,8.0,NL11_1_1,NL11_2000-EAG-2,"Boezem Amstelland-West, Noord-West",Amstellandboezem,None,None,NL_W11,"Waterschap Amstel, Gooi en Vecht",50000.0,...,M6b,R6,NaN,5.192,NaN,"Groot, ondiep kanaal dat vooral bestaat uit op...",11.0,825066.699,27279.918,"MULTIPOLYGON (((121854.85 478919.482, 121855.1..."
2,9.0,NL11_1_1,NL11_2000-EAG-4,"Boezem Amstelland-West, Oost",Amstellandboezem,None,None,NL_W11,"Waterschap Amstel, Gooi en Vecht",50000.0,...,M6b,R6,NaN,5.192,NaN,"Groot, ondiep kanaal dat vooral bestaat uit op...",11.0,664945.328,52456.598,"MULTIPOLYGON (((128211.113 468322.901, 128202...."
3,10.0,NL11_1_1,NL11_2000-EAG-3,"Boezem Amstelland-West, Noord-Oost",Amstellandboezem,None,None,NL_W11,"Waterschap Amstel, Gooi en Vecht",50000.0,...,M6b,R6,NaN,5.192,NaN,"Groot, ondiep kanaal dat vooral bestaat uit op...",11.0,536512.086,23164.338,"MULTIPOLYGON (((128179.03 483782.996, 128186.3..."
4,11.0,NL11_1_1,NL11_2000-EAG-6,"Boezem Amstelland-West, West",Amstellandboezem,None,None,NL_W11,"Waterschap Amstel, Gooi en Vecht",50000.0,...,M6b,R6,NaN,5.192,NaN,"Groot, ondiep kanaal dat vooral bestaat uit op...",11.0,1081855.826,52314.895,"MULTIPOLYGON (((119626.14 473859.15, 119623.87..."


Let's visualize the river using `geemap`.

In [ ]:
import geemap

# --- Convert Data for Earth Engine ---
# Convert the local GeoDataFrame into a cloud-based GEE FeatureCollection for server-side analysis.
amstel_fc = geemap.geopandas_to_ee(amstel_gdf)
# Extract a single, unified geometry from the FeatureCollection.
amstel_geom = amstel_fc.geometry()

# --- Calculate Map Center ---
# Calculate the centroid on GEE's servers, fetch the [lon, lat] result, and reverse it.
amstel_center = amstel_geom.centroid().coordinates().getInfo()[::-1]  # -> to [lat, lon] format

# --- Create a map centered on the Amstelland region ---
m = geemap.Map(center=amstel_center, zoom=11)

# --- Define a simple style for the river ---
river_style = {"color": "darkblue", "fillColor": "00000000", "width": 2}

# --- Add the styled river outline to the map ---
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# --- Display the map ---
m

### Part II: visualizing sampling locations for water quality

Now that we have our river system visualized, let's add the locations where water quality samples were taken. We will load data from two different monitoring networks that are used for lab analysis: **HB** (Hydrobiology) and **FYCHEM** (Physical and Chemical parameters), and display them as green and orange points on the map.

To do this, we'll use the `GeoJSON` layer from the `ipyleaflet` library. This is a great approach because **GeoJSON** is a lightweight, standard format and the native language for creating interactive vector layers on web maps. 🗺️

In [ ]:
from ipyleaflet import GeoJSON
import json

# --- Setup: Define colors and file paths ---
COLOR_HB = "green"
COLOR_FY = "orange"
hb_path = drive_folder / "waternet FEWS data/HB_unique_locations_with_measurements.geojson"
fy_path = drive_folder / "waternet FEWS data/FYCHEM_unique_locations_with_measurements.geojson"

# --- Data Loading: Read the geospatial data ---
hb_data = json.loads(gpd.read_file(hb_path).to_json())
fy_data = json.loads(gpd.read_file(fy_path).to_json())

# --- Map Creation ---
# Create the base map with the river outline
m = geemap.Map(center=amstel_center, zoom=11)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# Create simple, colored layers for the locations
hb_layer = GeoJSON(
    data=hb_data,
    name="HB Locations",
    point_style={'radius': 5, 'color': COLOR_HB, 'fillColor': COLOR_HB, 'fillOpacity': 0.7, 'weight': 1}
)
fy_layer = GeoJSON(
    data=fy_data,
    name="FYCHEM Locations",
    point_style={'radius': 5, 'color': COLOR_FY, 'fillColor': COLOR_FY, 'fillOpacity': 0.7, 'weight': 1}
)

# --- Add layers and display the final map ---
m.add_layer(hb_layer)
m.add_layer(fy_layer)
m

The map currently shows all available measurement locations, which can be a bit cluttered. To get a clearer view, let's sort the data and visualize only the sites with the most recorded observations. This will help us focus on the most frequently monitored areas in the Amstelland region. 📊

In [ ]:
# --- Helper Function: Get top K locations from a file ---
def get_top_k_gdf(path, k=10):
    """
    Reads a GeoJSON file, sorts it by 'total_observations',
    and returns the top 'k' entries as a GeoDataFrame.
    """
    return (
        gpd.read_file(path)
           .sort_values("total_observations", ascending=False)
           .head(k)
    )

# --- Load Top Locations & Prepare for Mapping ---
# Get the top 20 locations for each network
top_hb_gdf = get_top_k_gdf(hb_path, k=20)
top_fy_gdf = get_top_k_gdf(fy_path, k=20)

# --- Map Creation ---
# Create the base map with the river outline
m = geemap.Map(center=amstel_center, zoom=11)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# Create simple, colored layers for the top locations
# (convert the GeoDataFrames to GeoJSON-style dictionary, which is the format the map layer requires.)
hb_layer = GeoJSON(
    data=json.loads(top_hb_gdf.to_json()),
    name="Top HB Locations",
    point_style={'radius': 5, 'color': COLOR_HB, 'fillColor': COLOR_HB, 'fillOpacity': 0.7, 'weight': 1}
)
fy_layer = GeoJSON(
    data=json.loads(top_fy_gdf.to_json()),
    name="Top FYCHEM Locations",
    point_style={'radius': 5, 'color': COLOR_FY, 'fillColor': COLOR_FY, 'fillOpacity': 0.7, 'weight': 1}
)

# --- Add layers and display the final map ---
m.add_layer(hb_layer)
m.add_layer(fy_layer)
m

Looking at the map, we can see that some of our top measurement sites are located on the river, while others are further away. Although water from these outlying areas eventually drains into the Amstel, it can be useful to select locations within the river's immediate vicinity.

To demonstrate geospatial analysis, we will now perform a basic geometric operation. We'll create a "buffer" zone around the river and filter our selection to include only the top-ranked locations that fall within this specific perimeter. This is a common and powerful technique in geospatial data science. 🗺️

In [ ]:
# --- Helper Function Filter points near the river ---
def filter_points_near_river(points_gdf, river_gdf, distance_meters=20):
    """
    Selects points from points_gdf that are within a specified
    distance (in meters) of the river_gdf geometry.
    """
    # Use a projected CRS for accurate distance calculations in meters
    metric_crs = "EPSG:28992"

    # Re-project both GeoDataFrames to the metric CRS
    points_metric = points_gdf.to_crs(metric_crs)
    river_metric = river_gdf.to_crs(metric_crs)

    # Create a buffer polygon around the river
    river_buffer = river_metric.buffer(distance_meters).unary_union

    # Find the points that are within the buffer
    points_within_buffer = points_metric[points_metric.within(river_buffer)]

    # Convert the filtered points back to the original CRS for mapping
    return points_within_buffer.to_crs(points_gdf.crs)


# Filter those GeoDataFrames to find points near the river
#    (amstel_line_gdf was created in a previous cell)
near_river_hb_gdf = filter_points_near_river(top_hb_gdf, amstel_gdf, distance_meters=20)
near_river_fy_gdf = filter_points_near_river(top_fy_gdf, amstel_gdf, distance_meters=20)

# Convert the final, filtered GeoDataFrames to GeoJSON for the map ✅
hb_data = json.loads(near_river_hb_gdf.to_json())
fy_data = json.loads(near_river_fy_gdf.to_json())

# --- Map Creation ---
m = geemap.Map(center=amstel_center, zoom=11)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# Create layers for the filtered locations
hb_layer = GeoJSON(
    data=hb_data,
    name="Top HB Locations (Near River)",
    point_style={'radius': 5, 'color': COLOR_HB, 'fillColor': COLOR_HB, 'fillOpacity': 0.9, 'weight': 1}
)
fy_layer = GeoJSON(
    data=fy_data,
    name="Top FYCHEM Locations (Near River)",
    point_style={'radius': 5, 'color': COLOR_FY, 'fillColor': COLOR_FY, 'fillOpacity': 0.9, 'weight': 1}
)

# --- Add layers and display the final map ---
m.add_layer(hb_layer)
m.add_layer(fy_layer)
m

While visualizing the points is a good start, we need to be able to click on them to see their data. A robust way to handle many locations is to first create a `Marker` for each point individually, and then group them for display.

In this cell, we will loop through our **HB** and **FY** datasets, creating a unique marker for each station. We'll use the `HTML` widget from `ipywidgets` to format a `Popup` with key information for each point.

Finally, instead of adding these markers to the map one-by-one, we will add them to a `MarkerCluster`. This is a powerful feature that keeps the map clean by automatically grouping dense points together, which then expand as you zoom in. 🗺️

In [ ]:
from ipyleaflet import MarkerCluster, Popup, CircleMarker, LayersControl
from ipywidgets import HTML
import geemap

# --- Create the Map and Add the River Layer ---
m = geemap.Map(toolbar_ctrl=False)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# --- Manually Create Markers for Each Dataset  ---
hb_markers = []
for _, row in near_river_hb_gdf.iterrows():
    location = (row.geometry.y, row.geometry.x)
    popup_html = HTML(f"<b>Location Code:</b> {row['locatiecode']}<br><b>Total Obs:</b> {row['total_observations']}")
    marker = CircleMarker(
        location=location,
        radius=6,
        color="green",
        fill_color="green",
        fill_opacity=0.7,
        weight=1,
        popup=popup_html
    )
    hb_markers.append(marker)

fy_markers = []
for _, row in near_river_fy_gdf.iterrows():
    location = (row.geometry.y, row.geometry.x)
    popup_html = HTML(f"<b>Location Code:</b> {row['locatiecode']}<br><b>Total Obs:</b> {row['total_observations']}")
    marker = CircleMarker(
        location=location,
        radius=6,
        color="orange",
        fill_color="orange",
        fill_opacity=0.7,
        weight=1,
        popup=popup_html
    )
    fy_markers.append(marker)

# --- Create a Separate MarkerCluster for Each Dataset ---
# The 'name' argument here is specifically for the ipyleaflet LayersControl
hb_cluster = MarkerCluster(markers=hb_markers, name="HB Locations")
fy_cluster = MarkerCluster(markers=fy_markers, name="FYCHEM Locations")

# --- Add the Separate Clusters to the Map ---
m.add_layer(hb_cluster)
m.add_layer(fy_cluster)

# --- Add the LayersControl that can see your clusters ---
m.add_control(LayersControl(position='topright')) #

# --- Set Final View and Display ---
m.set_center(lon=amstel_center[1], lat=amstel_center[0], zoom=12)
m

### Part III: Pumping Stations



## Part III: The Pumping Stations (`Gemalen`)

In a highly engineered, low-lying landscape like the Amstelland polders, **pumping stations** (`gemalen`) play a vital role in mastering water levels and protecting the land. A single station often houses multiple pumps to ensure operational reliability (redundancy) and to provide scalable capacity for heavy rainfall.

- These gemalen lift excess water from the polders—a necessity in peat-based landscapes prone to subsidence and saturation—and move it into the **Amstel river**, which functions as a **`boezem`** (a managed basin of canals and lakes).

- The Amstel-boezem serves as a temporary storage reservoir, accommodating water from diverse polders before channeling it through Amsterdam and ultimately discharging it to the sea. This integration makes the **water quality of the Amstel** directly influenced by the varied origins of pumped water.

Understanding the geographic and functional placement of these gemalen is crucial—not only for assessing water quantity, but also for evaluating potential chemical and biological impacts on the Amstel river system.

In [ ]:
from ipyleaflet import Marker

# --- Load and Prepare Pumps Data ---
# Define path and read the shapefile into a GeoDataFrame
pumps_shp = drive_folder / "shapefiles/GEE (google earth engine)/gemalen_shapefile" / "poldergemalen_Amstelland.shp"
pumps_gdf = gpd.read_file(pumps_shp)

# Reproject to WGS84 (latitude/longitude) if necessary
if pumps_gdf.crs.to_epsg() != 4326:
    pumps_gdf = pumps_gdf.to_crs(epsg=4326)

# --- Create the Map ---
# IMPORTANT: This setup will likely cause the same layer control issue as before.
# See the note below for the best solution.
m = geemap.Map(toolbar_ctrl=False)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

# --- Manually Create a CircleMarker for Each Pumping Station ---
pump_markers = []
for _, row in pumps_gdf.iterrows():
    location = (row.geometry.y, row.geometry.x)
    popup_html = HTML(f"<b>ID:</b> {row.get('Locatie_id', 'N/A')}<br><b>Name:</b> {row.get('Locatie_na', 'N/A')}")

    # Create a styled CircleMarker instead of the default Marker
    marker = CircleMarker(
        location=location,
        radius=6,
        color="royalblue",       # Border color
        fill_color="cornflowerblue", # Fill color
        fill_opacity=0.8,
        weight=1,                # Border width
        popup=popup_html         # Attach popup directly
    )
    pump_markers.append(marker)

# --- Group All Markers into a Single Cluster ---
pumps_cluster = MarkerCluster(markers=pump_markers, name="Pumping Stations (Gemalen)")

# --- Add Cluster to the Map and Set Final View ---
m.add_layer(pumps_cluster)
m.set_center(lon=amstel_center[1], lat=amstel_center[0], zoom=11)

# --- Add the LayersControl that can see your clusters ---
m.add_control(LayersControl(position='topright')) #

m

You might have noticed **multiple pumps** at the same locations because of redundancy, but also because some pumps might have been replaced and old ones are still reported for completeness. The code below modifies the data frame to better illustrate these cases.

In [ ]:
# Split "Locatie_id" into location and pump identified
pumps_gdf[['unieke_locatie_id', 'pomp_id']] = pumps_gdf['Locatie_id'].str.split('_', n=1, expand=True)

# Show results
pumps_gdf[['Locatie_id', 'unieke_locatie_id', 'pomp_id']].head()

,Locatie_id,unieke_locatie_id,pomp_id
0,KGM00058_P002,KGM00058,P002
1,KGM00058_P001,KGM00058,P001
2,KGM00071_P003,KGM00071,P003
3,KGM00071_P002,KGM00071,P002
4,KGM00071_P001,KGM00071,P001


In [ ]:
pumps_gdf[pumps_gdf['unieke_locatie_id']=="KGM00103"][['Locatie_id', 'unieke_locatie_id', 'pomp_id']]

,Locatie_id,unieke_locatie_id,pomp_id
53,KGM00103_P002_Nieuw,KGM00103,P002_Nieuw
54,KGM00103_P001,KGM00103,P001
55,KGM00103_P001_Nieuw,KGM00103,P001_Nieuw


This final analysis shows the spatial relationship between polder pumping stations and water quality monitoring sensors within the Amstelland region. The primary goal is to identify which of these active sensors is located closest to each individual pumping station, providing a proxy for the quality of water being managed. The scope is limited to sensors from the FYCHEM network, and is further constrained to only the top 100 locations with the highest total number of historical observations

The process involves several key steps:
1.  **Nearest Neighbor Analysis:** A nearest-neighbor spatial join (`sjoin_nearest`) is used to programmatically link each pump to its single closest sensor from the prioritized list of the top 100 FYCHEM locations.
2.  **Visual Linking with Color:** To make these connections intuitive on a map, each pump and its corresponding sensor are assigned a unique, matching color. FYCHEM sensors that are not the closest to any pump are colored gray for contrast.
3.  **Resolving Overlaps:** In cases where multiple pumps exist at the exact same coordinates, a "jitter" algorithm is applied. This slightly offsets the map markers in a small circle, ensuring that all individual points remain visible and interactive.

The final output is an interactive map that visually clusters these relationships. This provides a clear tool for understanding the monitoring coverage of the pumping infrastructure, which is crucial as the polder water quality directly influences the larger *boezem* water system.

In [ ]:
import geopandas as gpd
import geemap
from ipyleaflet import MarkerCluster, Popup, CircleMarker
from ipywidgets import HTML
import matplotlib.pyplot as plt
import numpy as np

# --- Helper Function (assuming this is in a previous cell) ---
def get_top_k_gdf(path, k=100):
    return (
        gpd.read_file(path)
           .sort_values("total_observations", ascending=False)
           .head(k)
    )

# --- Data Loading and Geospatial Analysis ---
fy_top100_gdf = get_top_k_gdf(fy_path, k=100)
pumps_gdf = gpd.read_file(pumps_shp).to_crs(epsg=4326)

# Link each pump to its closest FYCHEM point
metric_crs = "EPSG:28992"
pumps_metric = pumps_gdf.to_crs(metric_crs)
fy_metric = fy_top100_gdf.to_crs(metric_crs)
pumps_linked_gdf = gpd.sjoin_nearest(pumps_metric, fy_metric, how='left')
pumps_linked_gdf = pumps_linked_gdf.to_crs(pumps_gdf.crs)

# Separate the FYCHEM points into "used" and "unused" groups
used_fychem_indices = pumps_linked_gdf['index_right'].unique()
fy_used_gdf = fy_top100_gdf.loc[used_fychem_indices].copy()
fy_unused_gdf = fy_top100_gdf.drop(used_fychem_indices).copy()

# --- Assign Colors ---

num_used_points = len(fy_used_gdf)
cmap = plt.cm.get_cmap('viridis', num_used_points)
colors = [f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' for r, g, b, a in cmap(np.linspace(0, 1, num_used_points))]
fy_used_gdf['link_color'] = colors

# Transfer the color from each used FYCHEM point to the pumps that link to it
pumps_linked_gdf = pumps_linked_gdf.merge(
    fy_used_gdf[['link_color']],
    left_on='index_right',
    right_index=True,
    how='left'
)

# --- Jitter Overlapping Pump Locations (NEW) ---

# Initialize final coordinates with original values
pumps_linked_gdf['final_lon'] = pumps_linked_gdf.geometry.x
pumps_linked_gdf['final_lat'] = pumps_linked_gdf.geometry.y

# Find geometries that appear more than once
counts = pumps_linked_gdf.geometry.value_counts()
overlapping_geoms = counts[counts > 1].index
jitter_radius = 0.00002  # Offset distance in degrees (approx. 5 meters)

# Iterate over only the locations that have overlapping points
for geom in overlapping_geoms:
    indices = pumps_linked_gdf[pumps_linked_gdf.geometry == geom].index
    num_pumps = len(indices)

    # Calculate new positions in a small circle around the original point
    for i, pump_index in enumerate(indices):
        angle = i * (2 * np.pi / num_pumps)
        dx = jitter_radius * np.cos(angle)
        pumps_linked_gdf.loc[pump_index, 'final_lon'] += dx


# --- Create Map and Markers ---

m = geemap.Map(toolbar_ctrl=False)
m.addLayer(amstel_fc.style(**river_style), {}, "Amstelland")

linked_markers = []
other_markers = []

# Create markers for the LINKED group using the new jittered coordinates
for _, row in pumps_linked_gdf.iterrows():
    marker = CircleMarker(
        location=(row['final_lat'], row['final_lon']),  # Use final coordinates
        radius=10, color=row['link_color'], weight=5, fill_color=row['link_color'], fill_opacity=0.2,
        popup=HTML(f"<b>Pump:</b> {row.get('Locatie_na', 'N/A')}<br><i>Links to: {row.get('locatiecode')}</i>")
    )
    linked_markers.append(marker)

for _, row in fy_used_gdf.iterrows():
    marker = CircleMarker(
        location=(row.geometry.y, row.geometry.x), radius=10, color=row['link_color'],
        weight=1, fill_color=row['link_color'], fill_opacity=0.8,
        popup=HTML(f"<b>FYCHEM:</b> {row.get('locatiecode', 'N/A')}<br><b>Obs:</b> {row.get('total_observations')}")
    )
    linked_markers.append(marker)

# Create markers for the OTHER group (Unused FYCHEM points)
for _, row in fy_unused_gdf.iterrows():
    marker = CircleMarker(
        location=(row.geometry.y, row.geometry.x), radius=5, color='#808080',
        weight=1, fill_color='#808080', fill_opacity=0.6,
        popup=HTML(f"<b>FYCHEM:</b> {row.get('locatiecode', 'N/A')}<br><b>Obs:</b> {row.get('total_observations')}")
    )
    other_markers.append(marker)


# --- Add Layers to Map and Display ---

linked_cluster = MarkerCluster(markers=linked_markers, name="Linked Pumps & Stations")
other_cluster = MarkerCluster(markers=other_markers, name="Other Top 100 Stations")

m.add_layer(linked_cluster)
m.add_layer(other_cluster)

m.set_center(lon=amstel_center[1], lat=amstel_center[0], zoom=11)

# --- Add the LayersControl that can see your clusters ---
m.add_control(LayersControl(position='topright')) #

m